In [2]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn import linear_model
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [3]:
df1 = pd.read_csv("Credit.csv", index_col = 0)
df1.head()

,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity,Balance
1,14.891,3606,283,2,34,11,Male,No,Yes,Caucasian,333
2,106.025,6645,483,3,82,15,Female,Yes,Yes,Asian,903
3,104.593,7075,514,4,71,11,Male,No,No,Asian,580
4,148.924,9504,681,3,36,11,Female,No,No,Asian,964
5,55.882,4897,357,2,68,16,Male,No,Yes,Caucasian,331


In [4]:
df = pd.get_dummies(df1, columns = ["Gender", "Student","Married","Ethnicity"], drop_first = True)
df.head()

,Income,Limit,Rating,Cards,Age,Education,Balance,Gender_Female,Student_Yes,Married_Yes,Ethnicity_Asian,Ethnicity_Caucasian
1,14.891,3606,283,2,34,11,333,0,0,1,0,1
2,106.025,6645,483,3,82,15,903,1,1,1,1,0
3,104.593,7075,514,4,71,11,580,0,0,0,1,0
4,148.924,9504,681,3,36,11,964,1,0,0,1,0
5,55.882,4897,357,2,68,16,331,0,0,1,0,1


In [5]:
n = len(df)
p = df.columns.tolist()
order = [0,1,2,3,4,5,7,8,9,10,11]
p = [p[i] for i in order]
df2 = df[p]
df2.head()

,Income,Limit,Rating,Cards,Age,Education,Gender_Female,Student_Yes,Married_Yes,Ethnicity_Asian,Ethnicity_Caucasian
1,14.891,3606,283,2,34,11,0,0,1,0,1
2,106.025,6645,483,3,82,15,1,1,1,1,0
3,104.593,7075,514,4,71,11,0,0,0,1,0
4,148.924,9504,681,3,36,11,1,0,0,1,0
5,55.882,4897,357,2,68,16,0,0,1,0,1


In [7]:
suma_p = {}
for i in p:
    if i != "Income":
        suma_p[i] = "+" + i
    else:
        suma_p[i] = i
formula = "Balance ~ "
pevaluados = []
for i in p:
    formula = formula + suma_p[i]
    print(formula)
    model = smf.ols(formula = formula, data = df)
    result = model.fit()
    pevaluados.append(i)
    print(pevaluados)
    print(result.pvalues)
    for j in pevaluados:
        if result.pvalues[j]>=0.08:
            print(suma_p[j])
            print(formula)
            formula = formula.replace(suma_p[j],"")
            pevaluados.remove(j)
            print(formula)
model = smf.ols(formula = formula, data = df)
result = model.fit()

Balance ~ Income
['Income']
Intercept    6.903440e-13
Income       1.030886e-22
dtype: float64
Balance ~ Income+Limit
['Income', 'Limit']
Intercept     3.878764e-61
Income        1.260933e-61
Limit        7.717386e-158
dtype: float64
Balance ~ Income+Limit+Rating
['Income', 'Limit', 'Rating']
Intercept    1.145371e-41
Income       7.979584e-64
Limit        5.896627e-02
Rating       6.106010e-05
dtype: float64
Balance ~ Income+Limit+Rating+Cards
['Income', 'Limit', 'Rating', 'Cards']
Intercept    4.346330e-42
Income       3.529576e-63
Limit        1.524244e-02
Rating       1.132753e-02
Cards        1.206030e-01
dtype: float64
+Cards
Balance ~ Income+Limit+Rating+Cards
Balance ~ Income+Limit+Rating
Balance ~ Income+Limit+Rating+Age
['Income', 'Limit', 'Rating', 'Age']
Intercept    1.278034e-24
Income       9.915136e-62
Limit        6.735675e-02
Rating       4.785510e-05
Age          7.430058e-02
dtype: float64
Balance ~ Income+Limit+Rating+Age+Education
['Income', 'Limit', 'Rating', 'Age

In [6]:
df_p = df[pevaluados]
x = df_p.loc[1,:]

In [7]:
model = smf.ols(formula = formula, data = df)
result = model.fit()
#result.predict(df_p)
formula

'Balance ~ Income+Limit+Rating+Age+Student_Yes'

In [8]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Balance   R-squared:                       0.953
Model:                            OLS   Adj. R-squared:                  0.952
Method:                 Least Squares   F-statistic:                     1585.
Date:                Sun, 29 Dec 2019   Prob (F-statistic):          2.25e-258
Time:                        18:07:30   Log-Likelihood:                -2409.4
No. Observations:                 400   AIC:                             4831.
Df Residuals:                     394   BIC:                             4855.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    -486.6144     25.289    -19.242      0.000    -536.333    -436.896
Income         -7.8722      0.238    -33.132      0.000      -8.339      -7.405
Limit           0.1197      0.028      4.307      0.000       0.065       0.174
Rating          2.2140      0.414      5.352      0.000       1.401       3.027
Age            -0.5761      0.298     -1.935      0.054      -1.161       0.009
Student_Yes   421.4457     16.827     25.045      0.000     388.363     454.528
==============================================================================
Omnibus:                       27.369   Durbin-Watson:                   1.909
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               31.721
Skew:                           0.690   Prob(JB):                     1.29e-07
Kurtosis:                       2.993   Cond. No.                     2.66e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.66e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [9]:
print(formula)
corr_o = df_p.corr()
corr = df_p.corr().values
corr = np.triu(corr)
for i in range(len(pevaluados)):
    for j in range(len(pevaluados)):
        if i != j:
            if corr[i][j]>=0.75:
                formula = formula.replace("+"+pevaluados[i],"")
print(formula)
corr

Balance ~ Income+Limit+Rating+Age+Student_Yes
Balance ~ Income+Rating+Age+Student_Yes


array([[ 1.        ,  0.79208834,  0.79137763,  0.1753384 ,  0.01963214],
       [ 0.        ,  1.        ,  0.99687974,  0.10088792, -0.00601509],
       [ 0.        ,  0.        ,  1.        ,  0.103165  , -0.00202765],
       [ 0.        ,  0.        ,  0.        ,  1.        , -0.02984443],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ]])

In [10]:
corr_o

,Income,Limit,Rating,Age,Student_Yes
Income,1.000000,0.792088,0.791378,0.175338,0.019632
Limit,0.792088,1.000000,0.996880,0.100888,-0.006015
Rating,0.791378,0.996880,1.000000,0.103165,-0.002028
Age,0.175338,0.100888,0.103165,1.000000,-0.029844
Student_Yes,0.019632,-0.006015,-0.002028,-0.029844,1.000000


In [17]:
model2 = smf.ols(formula = "Balance ~ Income + Limit + Income:Limit", data = df)
result2 = model2.fit()
result2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Balance   R-squared:                       0.875
Model:                            OLS   Adj. R-squared:                  0.874
Method:                 Least Squares   F-statistic:                     920.4
Date:                Sun, 29 Dec 2019   Prob (F-statistic):          4.54e-178
Time:                        18:27:33   Log-Likelihood:                -2604.2
No. Observations:                 400   AIC:                             5216.
Df Residuals:                     396   BIC:                             5232.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     -307.0868     30.459    -10.082      0.000    -366.968    -247.206
Income          -9.7852      0.746    -13.115      0.000     -11.252      -8.318
Limit            0.2524      0.007     36.906      0.000       0.239       0.266
Income:Limit     0.0003   8.08e-05      3.306      0.001       0.000       0.000
==============================================================================
Omnibus:                      106.052   Durbin-Watson:                   1.934
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              203.469
Skew:                           1.464   Prob(JB):                     6.56e-45
Kurtosis:                       4.906   Cond. No.                     1.76e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.76e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""